<a href="https://colab.research.google.com/github/hijyun/Project/blob/master/dacon/efficientNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install efficientnet_pytorch
!pip install git+https://github.com/cmpark0126/pytorch-polynomial-lr-decay.git

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.0-cp37-none-any.whl size=16031 sha256=dcf557769a8e197bb5498dafa95052c3e3827a6fe0943b2eba9f3ccb00e6439a
  Stored in directory: /root/.cache/pip/wheels/e9/c6/e1/7a808b26406239712cfce4b5ceeb67d9513ae32aa4b31445c6
Successfully built efficientnet-pytorch
  Cloning https://github.com/cmpark0126/pytorch-polynomial-lr-decay.git to /tmp/pip-req-build-cpq5k5yv
  Running command git clone -q https://github.com/cmpark0126/pytorch-polynomial-lr-decay.git /tmp/pip-req-build-cpq5k5yv
  Created wheel for torch-poly-lr-decay: filename=torch_poly_lr_decay-0.0.1-cp37-none-any.whl size=3833 sha256=d3a0e6eb50bbbc88c29e4ebc54846b98f7022bcc3a98681c1199228217be96ef
  Stored in directory: /tmp/pip-ephem-wheel-cache-ixkyc5s7/wheels/5a/b7/09/d748b20c9bdfc768a33c37a28b2ad7dd9ec3e79e5152cb1618
Successfully built torch-poly-lr-decay


In [ ]:
import torch
import glob
import os
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import cv2
from tqdm import tqdm
import numpy as np
from torchvision import transforms
import torchvision.models as models
import torch.nn as nn
from torch.nn import functional as F
from sklearn.model_selection import KFold
import time
from efficientnet_pytorch import EfficientNet
import matplotlib.pyplot as plt
from torch_poly_lr_decay import PolynomialLRDecay
import random

# Dacon Dataset Load

In [ ]:
torch.set_num_threads(1)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir "./dirty_mnist"

In [ ]:
!mkdir "./test_dirty_mnist"
!unzip "/content/drive/MyDrive/test_dirty_mnist_2nd.zip" -d "./test_dirty_mnist/"

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 extracting: ./test_dirty_mnist/50000.png  
 extracting: ./test_dirty_mnist/50001.png  
 extracting: ./test_dirty_mnist/50002.png  
 extracting: ./test_dirty_mnist/50003.png  
 extracting: ./test_dirty_mnist/50004.png  
 extracting: ./test_dirty_mnist/50005.png  
 extracting: ./test_dirty_mnist/50006.png  
 extracting: ./test_dirty_mnist/50007.png  
 extracting: ./test_dirty_mnist/50008.png  
 extracting: ./test_dirty_mnist/50009.png  
 extracting: ./test_dirty_mnist/50010.png  
 extracting: ./test_dirty_mnist/50011.png  
 extracting: ./test_dirty_mnist/50012.png  
 extracting: ./test_dirty_mnist/50013.png  
 extracting: ./test_dirty_mnist/50014.png  
 extracting: ./test_dirty_mnist/50015.png  
 extracting: ./test_dirty_mnist/50016.png  
 extracting: ./test_dirty_mnist/50017.png  
 extracting: ./test_dirty_mnist/50018.png  
 extracting: ./test_dirty_mnist/50019.png  
 extracting: ./test_dirty_mnist/50020.png  
 extracting: ./test_dirty_mnist/50021.pn

In [ ]:
!unzip "/content/drive/MyDrive/dirty_mnist_2nd.zip" -d "./dirty_mnist/"

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 extracting: ./dirty_mnist/45000.png  
 extracting: ./dirty_mnist/45001.png  
 extracting: ./dirty_mnist/45002.png  
 extracting: ./dirty_mnist/45003.png  
 extracting: ./dirty_mnist/45004.png  
 extracting: ./dirty_mnist/45005.png  
 extracting: ./dirty_mnist/45006.png  
 extracting: ./dirty_mnist/45007.png  
 extracting: ./dirty_mnist/45008.png  
 extracting: ./dirty_mnist/45009.png  
 extracting: ./dirty_mnist/45010.png  
 extracting: ./dirty_mnist/45011.png  
 extracting: ./dirty_mnist/45012.png  
 extracting: ./dirty_mnist/45013.png  
 extracting: ./dirty_mnist/45014.png  
 extracting: ./dirty_mnist/45015.png  
 extracting: ./dirty_mnist/45016.png  
 extracting: ./dirty_mnist/45017.png  
 extracting: ./dirty_mnist/45018.png  
 extracting: ./dirty_mnist/45019.png  
 extracting: ./dirty_mnist/45020.png  
 extracting: ./dirty_mnist/45021.png  
 extracting: ./dirty_mnist/45022.png  
 extracting: ./dirty_mnist/45023.png  
 extracting: ./dirty_mnist/4

In [ ]:
dirty_mnist_answer = pd.read_csv("/content/drive/MyDrive/dirty_mnist_2nd_answer.csv")
namelist = os.listdir('./dirty_mnist/')

In [ ]:
labels_df = pd.read_csv('/content/drive/MyDrive/dirty_mnist_2nd_answer.csv')[:]
imgs_dir = np.array(sorted(glob.glob('./dirty_mnist/*')))[:]
labels = np.array(labels_df.values[:,1:])

In [ ]:
test_imgs_dir = np.array(sorted(glob.glob('./test_dirty_mnist/*')))

In [ ]:
# 저장소에서 load
class MnistDataset_v1(Dataset):
    def __init__(self, imgs_dir=None, labels=None, transform=None, train=True):
        self.imgs_dir = imgs_dir
        self.labels = labels
        self.transform = transform
        self.train = train
        pass
    
    def __len__(self):
        # 데이터 총 샘플 수
        return len(self.imgs_dir)
    
    def __getitem__(self, idx):
        # 1개 샘플 get
        img = cv2.imread(self.imgs_dir[idx], cv2.IMREAD_COLOR)
        img = self.transform(img)
        if self.train==True:
            label = self.labels[idx]
            return img, label
        else:
            return img
        
        pass
    


# 메모리에서 load
class MnistDataset_v2(Dataset):
    def __init__(self, imgs=None, labels=None, transform=None, train=True):
        self.imgs = imgs
        self.labels = labels
        self.transform = transform
        self.train=train
        pass
    
    def __len__(self):
        # 데이터 총 샘플 수
        return len(self.imgs)
    
    def __getitem__(self, idx):
        # 1개 샘플 get1
        img = self.imgs[idx]
        img = self.transform(img)
        
        if self.train==True:
            label = self.labels[idx]
            return img, label
        else:
            return img
        
    

# reproduction을 위한 seed 설정

In [ ]:
# https://dacon.io/competitions/official/235697/codeshare/2363?page=1&dtype=recent&ptype=pub
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  

# model 정의

In [ ]:
# EfficientNet -b0(pretrained)
# MultiLabel output

class EfficientNet_MultiLabel(nn.Module):
    def __init__(self, in_channels):
        super(EfficientNet_MultiLabel, self).__init__()
        self.network = EfficientNet.from_pretrained('efficientnet-b0', in_channels=in_channels)
        self.output_layer = nn.Linear(1000, 26)

    def forward(self, x):
        x = F.relu(self.network(x))
        x = torch.sigmoid(self.output_layer(x))
        return x

# 데이터 분리

In [ ]:
# 해당 코드에서는 1fold만 실행

kf = KFold(n_splits=2, shuffle=True, random_state=42)
folds=[]
for train_idx, valid_idx in kf.split(imgs_dir):
    folds.append((train_idx, valid_idx))

# 훈련

In [ ]:
### seed_everything(42)


for fold in range(1):
    model = EfficientNet_MultiLabel(in_channels=3).to(device)
#     model = nn.DataParallel(model)
    train_idx = folds[fold][0]
    valid_idx = folds[fold][1]



    train_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip()

        ])
    valid_transform = transforms.Compose([
        transforms.ToTensor(),

        ])


    epochs=30
    batch_size=47         # 자신의 VRAM에 맞게 조절해야 OOM을 피할 수 있습니다.


    # Data Loader
    train_dataset = MnistDataset_v1(imgs_dir = imgs_dir[train_idx], labels=labels[train_idx], transform=train_transform)
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

    valid_dataset = MnistDataset_v1(imgs_dir = imgs_dir[valid_idx], labels = labels[valid_idx], transform=valid_transform)
    valid_loader = DataLoader(dataset=valid_dataset, batch_size=batch_size, shuffle=False)       
    
    
    # optimizer
    # polynomial optimizer를 사용합니다.
    # 
    optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
    decay_steps = (len(train_dataset)//batch_size)*epochs
    scheduler_poly_lr_decay = PolynomialLRDecay(optimizer, max_decay_steps=decay_steps, end_learning_rate=1e-6, power=0.9)
    #scheduler_poly_lr_decay = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=1, eta_min=0.001, last_epoch=-1)
    #scheduler_poly_lr_decay = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=0)
    criterion = torch.nn.BCELoss()
    
    
    epoch_accuracy = []
    valid_accuracy = []
    valid_losses=[]
    valid_best_accuracy=0
    for epoch in range(epochs):
        model.train()
        batch_accuracy_list = []
        batch_loss_list = []
        start=time.time()
        for n, (X, y) in enumerate((train_loader)):
            X = torch.tensor(X, device=device, dtype=torch.float32)
            y = torch.tensor(y, device=device, dtype=torch.float32)
            y_hat = model(X)
            
            
            optimizer.zero_grad()
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()
            scheduler_poly_lr_decay.step()

            
            y_hat  = y_hat.cpu().detach().numpy()
            y_hat = y_hat>0.5
            y = y.cpu().detach().numpy()

            batch_accuracy = (y_hat == y).mean()
            batch_accuracy_list.append(batch_accuracy)
            batch_loss_list.append(loss.item())

        model.eval()
        valid_batch_accuracy=[]
        valid_batch_loss = []
        with torch.no_grad():
            for n_valid, (X_valid, y_valid) in enumerate((valid_loader)):
                X_valid = torch.tensor(X_valid, device=device)#, dtype=torch.float32)
                y_valid = torch.tensor(y_valid, device=device, dtype=torch.float32)
                y_valid_hat = model(X_valid)
                
                valid_loss = criterion(y_valid_hat, y_valid).item()
                
                y_valid_hat = y_valid_hat.cpu().detach().numpy()>0.5
                
                
                valid_batch_loss.append(valid_loss)
                valid_batch_accuracy.append((y_valid_hat == y_valid.cpu().detach().numpy()).mean())
                
            valid_losses.append(np.mean(valid_batch_loss))
            valid_accuracy.append(np.mean(valid_batch_accuracy))
            
        if np.mean(valid_batch_accuracy)>valid_best_accuracy:
            torch.save(model.state_dict(), 'EfficientNetB0-fold{}.pt'.format(fold))
            valid_best_accuracy = np.mean(valid_batch_accuracy)
        print('fold : {}\tepoch : {:02d}\ttrain_accuracy / loss : {:.5f} / {:.5f}\tvalid_accuracy / loss : {:.5f} / {:.5f}\ttime : {:.0f}'.format(fold+1, epoch+1,
                                                                                                                                              np.mean(batch_accuracy_list),
                                                                                                                                              np.mean(batch_loss_list),
                                                                                                                                              np.mean(valid_batch_accuracy), 
                                                                                                                                              np.mean(valid_batch_loss),
                                                                                                                                              time.time()-start))

# Test Image 로드

In [ ]:
test_transform = transforms.Compose([
        transforms.ToTensor(),
        ])

# Test 추론

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/sample_submission.csv')

with torch.no_grad():
    for fold in range(1):
        model = EfficientNet_MultiLabel(in_channels=3).to(device)
        model.load_state_dict(torch.load('EfficientNetB0-fold{}.pt'.format(fold)))
        model.eval()

        test_dataset = MnistDataset_v1(imgs_dir = test_imgs_dir, transform=test_transform, train=False)
        test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

        for n, X_test in enumerate(tqdm(test_loader)):
            X_test = torch.tensor(X_test, device=device, dtype=torch.float32)
            with torch.no_grad():
                model.eval()
                pred_test = model(X_test).cpu().detach().numpy()
                submission.iloc[n*32:(n+1)*32,1:] += pred_test

# 제출물 생성

In [ ]:
submission.iloc[:,1:] = np.where(submission.values[:,1:]>=0.5, 1,0)

In [ ]:
submission.to_csv('EfficientNetB0-fold5.csv', index=False)

In [ ]:
from google.colab import files

files.download("EfficientNetB0-fold5-epoch15.csv")